# Algoritmo 3 - Associação de Regras

Vamos usar o Apriori para encontrar associações entre os desportos praticados pelos atletas olímpicos.

## Introdução
Este notebook aplica mineração de regras de associação sobre resultados olímpicos para identificar combinações relevantes entre modalidades e medalhas. Começamos pela consolidação dos ficheiros Summer e Winter, enriquecemos os registos com dados demográficos e, por fim, extraímos padrões frequentes com o algoritmo Apriori.

In [1]:
print("Hello World!")

# Instala/atualiza dependências usadas no notebook
%pip install --upgrade pip
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install mlxtend


Hello World!
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Preparação do ambiente
As linhas seguintes asseguram que todas as bibliotecas necessárias (pandas, numpy, matplotlib e mlxtend) estão instaladas e atualizadas. Executa-as apenas quando configurares o notebook pela primeira vez ou sempre que mudares de ambiente.

# Importar Ficheiro CSV
Carregamos os ficheiros `winter.csv` e `summer.csv`, adicionamos a coluna `Season` para distinguir as observações e fazemos o `concat` para ter um único dataset. Em seguida, renomeamos `Country` para `Country_Code` para alinhar com o dicionário e juntamos informação demográfica como população e PIB per capita.

In [2]:
import pandas as pd

# --- Carregar ---
winterData = pd.read_csv("Data/Olympics/winter.csv")
winterData["Season"] = "Winter"

summerData = pd.read_csv("Data/Olympics/summer.csv")
summerData["Season"] = "Summer"

# --- Limpeza "segura" por ficheiro (antes do concat) ---
TEXT_COLS = ["City","Sport","Discipline","Athlete","Country","Gender","Event","Medal"]

def clean_season_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates().copy()  # remove duplicados exatos
    for c in TEXT_COLS:
        df[c] = df[c].astype("string").str.strip()
    return df

winterData = clean_season_df(winterData)
summerData = clean_season_df(summerData)

# --- Unificar ---
concatData = pd.concat([winterData, summerData], ignore_index=True)

# Harmonizar código do país
concatData = concatData.rename(columns={"Country": "Country_Code"})

# --- Limpeza que precisa do dataset todo (depois do concat) ---

# 1) Remover linhas "Pending" (no teu summer.csv há 3)
concatData = concatData[concatData["Athlete"].ne("Pending")].copy()

# 2) Preencher Country_Code em falta pelo país mais frequente do mesmo atleta
#    (resolve o caso do KUDUKHOV, Besik em 2012)
mode_country = (
    concatData.dropna(subset=["Country_Code"])
              .groupby("Athlete")["Country_Code"]
              .agg(lambda x: x.mode().iat[0] if not x.mode().empty else pd.NA)
)
mask_missing = concatData["Country_Code"].isna()
concatData.loc[mask_missing, "Country_Code"] = concatData.loc[mask_missing, "Athlete"].map(mode_country)

# 3) (Opcional, mas recomendo) reduzir “inflacionamento” por equipas:
#    garante que cada (país, ano, season, sport, event, medal) conta 1 vez
concatData = concatData.drop_duplicates(
    subset=["Year","Season","Country_Code","Sport","Event","Medal"]
)

# 4) Normalização semântica (para consistência)
DISCIPLINE_MAP = {
    "Wrestling Free.": "Wrestling Freestyle",
    "Wrestling Gre-R": "Wrestling Greco-Roman",
    "Artistic G.": "Artistic Gymnastics",
    "Rhythmic G.": "Rhythmic Gymnastics",
    "Modern Pentath.": "Modern Pentathlon",
    "Synchronized S.": "Synchronized Swimming",
    "Beach volley.": "Beach Volleyball",
}
SPORT_MAP = {"Canoe": "Canoe / Kayak"}

concatData["Discipline"] = concatData["Discipline"].replace(DISCIPLINE_MAP)
concatData["Sport"] = concatData["Sport"].replace(SPORT_MAP)

# --- Dictionary: limpar e fazer merge com “bridge” de códigos ---
dictionary = pd.read_csv("Data/Olympics/dictionary.csv").copy()
dictionary["Country"] = dictionary["Country"].astype("string").str.replace("*", "", regex=False).str.strip()

# Alguns códigos do dataset olímpico não existem no dictionary com o mesmo nome
NOC_TO_DICT = {
    "ROU": "ROM",  # Romania
    "SGP": "SIN",  # Singapore
    "TTO": "TRI",  # Trinidad and Tobago
    "SRB": "SCG",  # (Serbia está como SCG)
}
concatData["DictCode"] = concatData["Country_Code"].replace(NOC_TO_DICT)

olympics = concatData.merge(
    dictionary[["Code", "Country", "Population", "GDP per Capita"]],
    left_on="DictCode",
    right_on="Code",
    how="left"
)


# se juntar falhar, isto evita perder o país no groupby
olympics["Country_Final"] = olympics["Country"].fillna(olympics["Country_Code"])


## Construir transações
Criamos a variável `SportMedal` para combinar desporto e medalha e agregamos por país/ano/temporada para obter o conjunto de itens por participação olímpica. Apenas mantemos transações com pelo menos um item antes de aplicar o `TransactionEncoder`.

In [3]:
from mlxtend.preprocessing import TransactionEncoder


# Criar item combinando desporto e medalha

medal_df = olympics[olympics["Medal"].isin(["Gold","Silver","Bronze"])].copy()

# itens = desportos onde o país medalhou naquele ano/season
transactions_df = (
    medal_df
    .groupby(["Country_Final", "Year", "Season"])["Sport"]
    .unique()
    .reset_index()
)

transactions_df["num_items"] = transactions_df["Sport"].apply(len)
transactions_df = transactions_df[transactions_df["num_items"] >= 1].reset_index(drop=True)

transactions = transactions_df["Sport"].apply(list).tolist()

# Transformar para matriz one-hot
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_ary, columns=te.columns_)


## Descobrir regras de associação
Aplicamos o `apriori` para gerar `frequent_itemsets` com suporte mínimo de 2% e, em seguida, calculamos regras com confiança mínima de 0.5. Filtramos as regras simples (1 antecedente → 1 consequente) com suporte relevante e `lift` > 1.5 para destacar relações realmente interessantes entre desporto e medalha.

In [4]:
from mlxtend.frequent_patterns import apriori, association_rules

# (Sugestão) Para multi-antecedente, muitas vezes precisas de suporte um pouco mais baixo
MIN_SUPPORT = 0.02
MIN_CONF = 0.5
MIN_LIFT_SIMPLE = 1.5
MIN_LIFT_MULTI = 1.2

# Controla complexidade: max_len=4 permite antecedentes até 3 (se consequente for 1)
frequent_itemsets = apriori(
    basket,
    min_support=MIN_SUPPORT,
    use_colnames=True,
    max_len=4
)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONF)

# calcular comprimentos (uma vez só)
rules["antecedent_len"] = rules["antecedents"].apply(len)
rules["consequent_len"] = rules["consequents"].apply(len)

# --- 1) REGRAS SIMPLES (1 -> 1) ---
simple_rules = (
    rules[
        (rules["antecedent_len"] == 1) &
        (rules["consequent_len"] == 1) &
        (rules["support"] >= MIN_SUPPORT) &
        (rules["lift"] >= MIN_LIFT_SIMPLE)
    ]
    .sort_values(["lift", "confidence", "support"], ascending=False)
)

# --- 2) REGRAS COM MAIS DO QUE UM ANTECEDENTE (k -> 1, k>=2) ---
multi_antecedent_rules = (
    rules[
        (rules["antecedent_len"] >= 2) &
        (rules["consequent_len"] == 1) &
        (rules["support"] >= MIN_SUPPORT) &
        (rules["lift"] >= MIN_LIFT_MULTI)
    ]
    .sort_values(["lift", "confidence", "support"], ascending=False)
)

# antecedente no max 3
multi_antecedent_rules = multi_antecedent_rules[multi_antecedent_rules["antecedent_len"] <= 3]


print("\n=== Regras multi-antecedente (2+ -> 1) ===")
multi_antecedent_rules



=== Regras multi-antecedente (2+ -> 1) ===


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,antecedent_len,consequent_len
3120,"(Judo, Shooting, Gymnastics)",(Archery),0.040365,0.046875,0.020182,0.500000,10.666667,1.0,0.018290,1.906250,0.944369,0.300971,0.475410,0.465278,3,1
2419,"(Weightlifting, Canoe / Kayak, Aquatics)",(Modern Pentathlon),0.037760,0.050130,0.020182,0.534483,10.661890,1.0,0.018289,2.040461,0.941770,0.298077,0.509915,0.468540,3,1
2792,"(Weightlifting, Aquatics, Fencing)",(Modern Pentathlon),0.042318,0.050130,0.021484,0.507692,10.127473,1.0,0.019363,1.929423,0.941083,0.302752,0.481710,0.468132,3,1
5261,"(Canoe / Kayak, Wrestling, Fencing)",(Modern Pentathlon),0.044922,0.050130,0.022786,0.507246,10.118577,1.0,0.020535,1.927677,0.943558,0.315315,0.481241,0.480896,3,1
2321,"(Canoe / Kayak, Aquatics, Fencing)",(Modern Pentathlon),0.046875,0.050130,0.023438,0.500000,9.974026,1.0,0.021088,1.899740,0.943989,0.318584,0.473612,0.483766,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,"(Weightlifting, Boxing)",(Athletics),0.087891,0.397786,0.065104,0.740741,1.862157,1.0,0.030142,2.322824,0.507602,0.154799,0.569489,0.452203,2,1
696,"(Judo, Weightlifting)",(Athletics),0.049479,0.397786,0.036458,0.736842,1.852356,1.0,0.016776,2.288411,0.484100,0.088748,0.563016,0.414248,2,1
563,"(Wrestling, Boxing)",(Athletics),0.130208,0.397786,0.093099,0.715000,1.797447,1.0,0.041304,2.113030,0.510071,0.214072,0.526746,0.474521,2,1
752,"(Weightlifting, Wrestling)",(Athletics),0.105469,0.397786,0.072266,0.685185,1.722495,1.0,0.030312,1.912914,0.468901,0.167674,0.477237,0.433427,2,1


In [5]:
# --- Output ---
print("=== Regras simples (1 -> 1) ===")
print(simple_rules.head(20))


=== Regras simples (1 -> 1) ===
             antecedents      consequents  antecedent support  \
102         (Volleyball)           (Judo)            0.046875   
105               (Luge)         (Skiing)            0.034505   
23             (Archery)           (Judo)            0.046875   
55            (Biathlon)         (Skiing)            0.055990   
85   (Modern Pentathlon)        (Fencing)            0.050130   
22             (Archery)     (Gymnastics)            0.046875   
71   (Modern Pentathlon)  (Canoe / Kayak)            0.050130   
101         (Ice Hockey)         (Skiing)            0.049479   
100         (Ice Hockey)        (Skating)            0.049479   
94          (Volleyball)     (Gymnastics)            0.046875   
21             (Archery)        (Fencing)            0.046875   
104               (Luge)        (Skating)            0.034505   
70            (Handball)  (Canoe / Kayak)            0.038411   
54            (Biathlon)        (Skating)            0.055

## Interpretar resultados
O dataframe `simple_rules` contém as associações mais fortes (lift > 1.5) entre desporto e medalha, enquanto `rules_sorted` lista o top 20 ordenado por `lift`. Analisa estas tabelas para identificar padrões consistentes (por exemplo, modalidades que costumam resultar em determinadas medalhas) e decide quais regras merecem destaque no relatório final.

## Conclusão
O pipeline consolidado permitiu descrever o panorama olímpico ao nível de país, edição e temporada, revelando combinações de desporto-medalha com padrões consistentes. As regras com maior lift ajudam a direcionar a análise para modalidades onde a probabilidade de medalha é mais elevada, servindo de base para recomendações estratégicas ou storytelling no relatório final. Ajusta os limiares de suporte/confiança conforme necessário para aprofundar segmentos específicos ou testar hipóteses adicionais.